In [7]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import pandas as pd
from sklearn import model_selection
from sklearn.svm import SVC

train_df = pd.read_csv("D:/Code/VSCode_Python/data/titanic_data/train.csv")
print(train_df.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [8]:
# 特征工程
clf = print(train_df.info())
train_df.loc[train_df.Embarked.isnull(), "Embarked"] = train_df.Embarked.mode()[0]
train_df.Cabin.fillna(
    "Unkonwn", inplace=True
)  # train_df.Age.fillna(train_df.Age.median(), inplace=True)
train_df.Cabin.apply(lambda x: x[0])
tg = train_df.Ticket.value_counts()  # 同行人
train_df["Ticket_group"] = train_df.Ticket.apply(lambda x: tg[x])
train_df["f_name"] = train_df.Name.apply(lambda x: x[0 : x.find(",")])
train_df["title"] = train_df.Name.apply(lambda x: x[x.find(",") + 1 : x.find(".")])
train_df["l_name"] = train_df.Name.apply(lambda x: x[x.find(".") + 1 :])

titles = train_df["title"].unique()
for title in titles:
    m_age = train_df.loc[train_df.title == title, "Age"].median()
    train_df.loc[(train_df.Age.isnull()) & (train_df.title == title), "Age"] = m_age
print("---特征工程：train_df.info----")
print(train_df.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
---特征工程：train_df.info----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   891

In [9]:
# 连续数值型的转换
# ------family_size--------
train_df["family_size"] = train_df.Parch + train_df.SibSp + 1

df2 = train_df[["family_size", "Survived"]].groupby(["family_size"]).mean()
train_df["fs_type"] = "a"
train_df.loc[(train_df.family_size > 1) & (train_df.family_size < 4), "fs_type"] = "b"
train_df.loc[(train_df.family_size >= 4), "fs_type"] = "c"
# ------family_size------end

# ------Age------------
df2 = train_df[["Age", "Survived"]].groupby(["Age"])
train_df["age_type"] = "a"
train_df.loc[(train_df.Age > 16) & (train_df.Age < 40), "age_type"] = "b"
train_df.loc[(train_df.Age >= 40), "age_type"] = "c"

# ------Age------------end

# ------Fare----------
df2 = train_df[["Fare", "Survived"]].groupby(["Fare"]).mean()

train_df["fare_type"] = "a"
train_df.loc[(train_df.Fare > 13) & (train_df.Fare <= 25), "fare_type"] = "b"
train_df.loc[(train_df.Fare > 25) & (train_df.Fare <= 38), "fare_type"] = "c"
train_df.loc[(train_df.Fare > 38) & (train_df.Fare <= 52), "fare_type"] = "d"
train_df.loc[(train_df.Fare > 52) & (train_df.Fare <= 102), "fare_type"] = "e"
train_df.loc[(train_df.Fare > 102), "fare_type"] = "f"

# ------Fare----------end


In [10]:
# 加一个child字段，对结果几乎没有帮助?
# train_df.loc[:,'child'] = 0
# train_df.loc[train_df.Age>=18, 'child']=1

# 是否有家人遇难，是否有家人获救
train_df["Family_Survival"] = 0.5
for grp, grp_df in train_df[
    [
        "Survived",
        "Name",
        "f_name",
        "Fare",
        "Ticket",
        "PassengerId",
        "SibSp",
        "Parch",
        "Age",
        "Cabin",
    ]
].groupby(["f_name", "Fare"]):

    if len(grp_df) != 1:
        for ind, row in grp_df.iterrows():
            # 删除自己遇难信息
            smax = grp_df.drop(ind)["Survived"].max()
            smin = grp_df.drop(ind)["Survived"].min()
            passID = row["PassengerId"]
            if smax == 1.0:
                train_df.loc[train_df["PassengerId"] == passID, "Family_Survival"] = 1
            elif smin == 0.0:
                train_df.loc[train_df["PassengerId"] == passID, "Family_Survival"] = 0

In [11]:
# 选择算法交叉
# 划分特征数据和标签数据，去除不需要的列
train_df = train_df.drop(
    ["Age", "Fare", "Parch", "SibSp", "family_size", "f_name", "l_name"], axis=1
)
X = train_df.iloc[:, 2:]
y = train_df.iloc[:, 1]

X = X.drop("Name", axis=1)
X = pd.get_dummies(X)

# 定义一个算法,建立模型和验证模型
# clf = DecisionTreeClassifier()  # 1 dtreep
# clf1 = RandomForestClassifier()  #  2 随机森林
# clfk = KNeighborsClassifier(n_neighbors=3)  # 4 k临近

clf2 = SVC(kernel="linear", C=5)  # 3 支持向量积
# 逻辑回归
clf3 = LogisticRegression(max_iter=1000)
# 神经网络
clf4 = MLPClassifier(
    max_iter=1000, hidden_layer_sizes=(100, 60), alpha=0.1
)  # 设置感知器，中间是隐藏层

# 特征选择
X = SelectKBest(chi2, k=700).fit_transform(X, y)

In [12]:
# 集成3个算法
vof = VotingClassifier([("clf2", clf2), ("clf3", clf3), ("clf4", clf4)])

# 交叉检验
sc = model_selection.cross_val_score(vof, X, y, scoring="accuracy")

print(sc.mean())

0.8619295712761282
